In [11]:
import pandas as pd
import numpy as np
from urllib.parse import quote
import webbrowser
from tqdm import tqdm

from vietunits.utils import geolocator, check_point_in_polygon, find_nearest_point

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/danhmuc_and_sapnhap.csv')

In [3]:
df.shape

(10602, 19)

In [4]:
# Các ward bị chia
df_divided = df[df['isDividedWard']==True].copy()
df_divided.sort_values(by=['province', 'district', 'ward', 'newWardAreaKm2'], inplace=True)

In [5]:
df_divided

,provinceCode,isMergedProvince,districtCode,districtType,wardCode,wardType,isMergedWard,isDividedWard,province,district,ward,newProvince,newWard,newProvinceCode,newWardCode,newWardType,newWardLat,newWardLon,newWardAreaKm2
10208,92,True,927,Huyện,31268.0,Xã,True,True,Thành phố Cần Thơ,Huyện Thới Lai,Xã Tân Thạnh,Thành phố Cần Thơ,Trường Thành,33,3275.0,xã,10.04230,105.610,59.09
10207,92,True,927,Huyện,31268.0,Xã,True,True,Thành phố Cần Thơ,Huyện Thới Lai,Xã Tân Thạnh,Thành phố Cần Thơ,Tân Thạnh,33,3254.0,xã,9.62301,106.064,70.80
10185,92,True,924,Huyện,31246.0,Xã,True,True,Thành phố Cần Thơ,Huyện Vĩnh Thạnh,Xã Thạnh Quới,Thành phố Cần Thơ,Gia Hòa,33,3204.0,xã,9.43175,105.800,77.06
10186,92,True,924,Huyện,31246.0,Xã,True,True,Thành phố Cần Thơ,Huyện Vĩnh Thạnh,Xã Thạnh Quới,Thành phố Cần Thơ,Thạnh Quới,33,3258.0,xã,10.20750,105.348,103.86
10155,92,True,918,Quận,31178.0,Phường,True,True,Thành phố Cần Thơ,Quận Bình Thủy,Phường Bùi Hữu Nghĩa,Thành phố Cần Thơ,Cái Khế,33,3193.0,phường,10.05170,105.781,10.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9791,87,True,874,Huyện,30157.0,Xã,True,True,Tỉnh Đồng Tháp,Huyện Thanh Bình,Xã Tân Thạnh,Tỉnh Đồng Tháp,Thanh Bình,31,3069.0,xã,10.41230,106.386,86.00
9699,87,True,867,Thành phố,29902.0,Phường,True,True,Tỉnh Đồng Tháp,Thành phố Sa Đéc,Phường 3,Tỉnh Đồng Tháp,Sa Đéc,31,3046.0,phường,10.32900,105.739,46.90
9700,87,True,867,Thành phố,29902.0,Phường,True,True,Tỉnh Đồng Tháp,Thành phố Sa Đéc,Phường 3,Tỉnh Đồng Tháp,Cao Lãnh,31,2997.0,phường,10.43990,105.624,73.30
9702,87,True,867,Thành phố,29908.0,Phường,True,True,Tỉnh Đồng Tháp,Thành phố Sa Đéc,Phường 4,Tỉnh Đồng Tháp,Sa Đéc,31,3046.0,phường,10.32900,105.739,46.90


# Default newWard when street is not provided

In [6]:
df_divided_ward = df_divided[['province', 'district', 'ward']].drop_duplicates()

In [12]:
# for index, row in tqdm(df_divided_ward.iterrows(), total=len(df_divided_ward)):
#     address = row['ward'] + ', ' + row['district'] + ', ' + row['province']
#     location = geolocator.geocode(address)
#     df_divided_ward.loc[index, ['wardLat', 'wardLon', 'wardGeoAddress']] = location.latitude, location.longitude, location.address
# df_divided_ward.to_csv('data/df_divided_ward.csv', index=False)
df_divided_ward = pd.read_csv('data/df_divided_ward.csv')

100%|██████████| 471/471 [05:52<00:00,  1.34it/s]


In [ ]:
for index, row in df_divided_ward.iterrows():
    province = row['province']
    district = row['district']
    ward = row['ward']
    ward_point = (row['wardLat'], row['wardLon'])

    new_wards = df[(df['province']==province) & (df['district']==district) & (df['ward']==ward)]

    containing_points= []
    new_ward_points = []
    for new_ward_index, new_ward_row in new_wards.iterrows():
        new_ward_point = (new_ward_row['newWardLat'], new_ward_row['newWardLon'])
        new_ward_area_km2 = new_ward_row['newWardAreaKm2']
        new_ward_points.append(new_ward_point)

        is_contain = check_point_in_polygon(point=ward_point, polygon_center=new_ward_point, polygon_area_km2=new_ward_area_km2)
        if is_contain:
            containing_points.append(new_ward_point)
        df.loc[new_ward_index, 'isNewWardPolygonContainsWard'] = is_contain

    nearest_point = find_nearest_point(a_point=ward_point, list_of_b_points=new_ward_points)

    if len(containing_points) == 1:
        default_ward_point = containing_points[0]
    else:
        default_ward_point = nearest_point


    df.loc[(df['province']==province) & (df['district']==district) & (df['ward']==ward) & (df['newWardLat']==nearest_point[0])& (df['newWardLon']==nearest_point[1]), 'isNearestNewWard'] = True
    df.loc[(df['province']==province) & (df['district']==district) & (df['ward']==ward) & (df['newWardLat']==default_ward_point[0])& (df['newWardLon']==default_ward_point[1]), 'isDefaultNewWard'] = True

df.loc[(df['isDividedWard']==True) & (df['isNearestNewWard'].isna()), 'isNearestNewWard'] = False
df.loc[(df['isDividedWard']==True) & (df['isDefaultNewWard'].isna()), 'isDefaultNewWard'] = False

In [23]:
df.to_csv('data/danhmuc_and_sapnhap_has_default_new_ward.csv', index=False)

# Allocate streets to new ward

In [26]:
# Các district có ward bị chia
df_divided_district = df_divided[['province', 'district']].drop_duplicates()
#
# # # Tìm tên đường phố thủ công
# # for index, row in tqdm(df_divided_district.iterrows(), total=df_divided_district.shape[0]):
# #     keyword = row['district'] + ', ' + row['province'] + ' wikipedia'
# #     search_url = f"https://www.google.com/search?q={quote(keyword)}"
# #     webbrowser.open(search_url)
# #     wikipedia = input("wikipedia:").upper()
# #     if wikipedia != 'NO':
# #         element = input("element:")
# #     else:
# #         element = 'NO'
# #     df_divided_district.loc[index, 'wikipedia'] = wikipedia
# #     df_divided_district.loc[index, 'element'] = element
# # df_divided_district.to_csv('data/df_divided_district.csv', index=False)
#
df_divided_district_2 = pd.read_csv('data/df_divided_district.csv')

In [25]:
df_divided_district

,province,district
10208,Thành phố Cần Thơ,Huyện Thới Lai
10185,Thành phố Cần Thơ,Huyện Vĩnh Thạnh
10155,Thành phố Cần Thơ,Quận Bình Thủy
10166,Thành phố Cần Thơ,Quận Thốt Nốt
488,Thành phố Hà Nội,Huyện Ba Vì
...,...,...
8550,Tỉnh Đồng Nai,Huyện Xuân Lộc
9763,Tỉnh Đồng Tháp,Huyện Cao Lãnh
9734,Tỉnh Đồng Tháp,Huyện Hồng Ngự
9790,Tỉnh Đồng Tháp,Huyện Thanh Bình


In [27]:
df_divided_district_2

,province,district,wikipedia,element
0,Thành phố Cần Thơ,Huyện Cờ Đỏ,https://vi.wikipedia.org/wiki/C%E1%BB%9D_%C4%9...,"Hà Huy Giáp,Lê Đức Thọ,Nguyễn Trọng Quyền,Nguy..."
1,Thành phố Cần Thơ,Huyện Thới Lai,NO,NO
2,Thành phố Cần Thơ,Huyện Vĩnh Thạnh,https://vi.wikipedia.org/wiki/V%C4%A9nh_Th%E1%...,Phù Đổng Thiên Vương
3,Thành phố Cần Thơ,Quận Bình Thủy,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,Cách mạng Tháng 8\nLê Hồng Phong\nNguyễn Văn L...
4,Thành phố Cần Thơ,Quận Thốt Nốt,https://vi.wikipedia.org/wiki/Th%E1%BB%91t_N%E...,30 tháng 4\nBạch Đằng\nHòa Bình\nLê Lợi\nLê Th...
...,...,...,...,...
281,Tỉnh Đồng Tháp,Huyện Hồng Ngự,NO,NO
282,Tỉnh Đồng Tháp,Huyện Lai Vung,NO,NO
283,Tỉnh Đồng Tháp,Huyện Thanh Bình,NO,NO
284,Tỉnh Đồng Tháp,Huyện Tháp Mười,NO,NO


In [29]:
pd.merge(df_divided_district, df_divided_district_2, on=['province', 'district'], how='left')

,province,district,wikipedia,element
0,Thành phố Cần Thơ,Huyện Thới Lai,NO,NO
1,Thành phố Cần Thơ,Huyện Vĩnh Thạnh,https://vi.wikipedia.org/wiki/V%C4%A9nh_Th%E1%...,Phù Đổng Thiên Vương
2,Thành phố Cần Thơ,Quận Bình Thủy,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,Cách mạng Tháng 8\nLê Hồng Phong\nNguyễn Văn L...
3,Thành phố Cần Thơ,Quận Thốt Nốt,https://vi.wikipedia.org/wiki/Th%E1%BB%91t_N%E...,30 tháng 4\nBạch Đằng\nHòa Bình\nLê Lợi\nLê Th...
4,Thành phố Hà Nội,Huyện Ba Vì,https://vi.wikipedia.org/wiki/Ba_V%C3%AC_(huy%...,Chùa Cao\nCống Ải\nĐá Bạc\nĐông Hưng\nGò Sóc\n...
...,...,...,...,...
160,Tỉnh Đồng Nai,Huyện Xuân Lộc,NO,NO
161,Tỉnh Đồng Tháp,Huyện Cao Lãnh,NO,NO
162,Tỉnh Đồng Tháp,Huyện Hồng Ngự,NO,NO
163,Tỉnh Đồng Tháp,Huyện Thanh Bình,NO,NO


# Không có street, default